# 🖼️ RPA自動キャプチャテストシステム

アプリ再起動時に自動で画面をキャプチャして指定場所に保存するシステム

## 📋 機能概要
- ✅ 自動スクリーンショット取得
- ✅ 指定フォルダへの自動保存
- ✅ タイムスタンプ付きファイル名
- ✅ エラーハンドリング
- ✅ 履歴管理

In [1]:
# 必要ライブラリのインストール・インポート
import sys
import os
import asyncio
from datetime import datetime
from pathlib import Path
import time
import json

# プロジェクトルートをパスに追加
sys.path.append('/workspaces/fastapi_django_main_live')

print("📚 ライブラリ読み込み完了")
print(f"⏰ 実行開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📚 ライブラリ読み込み完了
⏰ 実行開始時刻: 2025-06-13 21:22:09


In [2]:
# RPAモジュールの読み込み
try:
    from contbk.gra_12_rpa.rpa_automation import RPAManager
    RPA_AVAILABLE = True
    print("✅ RPA機能利用可能")
except ImportError as e:
    RPA_AVAILABLE = False
    print(f"❌ RPA機能利用不可: {e}")
    print("📦 Playwrightのインストールが必要な場合があります")

❌ RPA機能利用不可: No module named 'contbk'
📦 Playwrightのインストールが必要な場合があります


In [3]:
# キャプチャ設定
class CaptureConfig:
    """キャプチャ設定クラス"""
    
    def __init__(self):
        # 保存先ディレクトリ
        self.save_dir = Path("/workspaces/fastapi_django_main_live/docs/images/auto_captures")
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        # キャプチャ対象URL（Codespace URL）
        self.target_urls = [
            "https://ideal-halibut-4q5qp79g2jp9-7860.app.github.dev/",
            "https://ideal-halibut-4q5qp79g2jp9-7860.app.github.dev/?tab=0",  # 最初のタブ
            "https://ideal-halibut-4q5qp79g2jp9-7860.app.github.dev/?tab=1",  # 2番目のタブ
        ]
        
        # セレクター指定キャプチャ
        self.selector_captures = {
            "main_interface": ".gradio-container",
            "tabs_area": ".tab-nav",
            "rpa_debug_tab": "button[data-testid='🔍 RPA + AI デバッグ-tab-button']"
        }
        
        # 待機時間設定
        self.wait_time = 5  # 画面読み込み待機秒数
        self.capture_interval = 2  # キャプチャ間隔秒数
        
        # ファイル名形式
        self.filename_format = "capture_{timestamp}_{type}.png"
        
        # 履歴保存ファイル
        self.history_file = self.save_dir / "capture_history.json"
        
        print(f"💾 保存先: {self.save_dir}")
        print(f"🌐 対象URL数: {len(self.target_urls)}")
        print(f"🎯 セレクター数: {len(self.selector_captures)}")

# 設定インスタンス作成
config = CaptureConfig()
print("⚙️ キャプチャ設定完了")

💾 保存先: /workspaces/fastapi_django_main_live/docs/images/auto_captures
🌐 対象URL数: 3
🎯 セレクター数: 3
⚙️ キャプチャ設定完了


In [4]:
# キャプチャ実行クラス
class AutoCaptureSystem:
    """自動キャプチャシステム"""
    
    def __init__(self, config):
        self.config = config
        self.rpa_manager = RPAManager() if RPA_AVAILABLE else None
        self.capture_results = []
        
    async def capture_single_url(self, target_info):
        """単一URLのキャプチャ"""
        name = target_info["name"]
        url = target_info["url"]
        prefix = target_info["filename_prefix"]
        wait_time = target_info["wait_time"]
        
        print(f"\n📸 キャプチャ開始: {name}")
        print(f"🌐 URL: {url}")
        
        try:
            if not self.rpa_manager:
                return {
                    "name": name,
                    "success": False,
                    "error": "RPA機能が利用できません"
                }
            
            # スクリーンショット取得
            img, message = await self.rpa_manager.capture_screenshot(
                url=url,
                selector=None,
                wait_time=wait_time
            )
            
            if img:
                # ファイル名生成
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"{prefix}_{timestamp}.png"
                filepath = self.config.save_dir / filename
                
                # 画像保存
                img.save(filepath)
                
                result = {
                    "name": name,
                    "success": True,
                    "filepath": str(filepath),
                    "filename": filename,
                    "timestamp": datetime.now().isoformat(),
                    "url": url,
                    "message": message
                }
                
                print(f"✅ 保存完了: {filename}")
                return result
            else:
                print(f"❌ キャプチャ失敗: {message}")
                return {
                    "name": name,
                    "success": False,
                    "error": message
                }
                
        except Exception as e:
            error_msg = f"エラー: {str(e)}"
            print(f"❌ {error_msg}")
            return {
                "name": name,
                "success": False,
                "error": error_msg
            }
    
    async def capture_all_urls(self):
        """全URLのキャプチャ実行"""
        print("🚀 一括キャプチャ開始")
        print("=" * 50)
        
        results = []
        
        for target_info in self.config.target_urls:
            result = await self.capture_single_url(target_info)
            results.append(result)
            
            # 次のキャプチャまで少し待機
            await asyncio.sleep(2)
        
        self.capture_results = results
        return results
    
    def save_history(self, results):
        """履歴をJSONファイルに保存"""
        try:
            # 既存履歴を読み込み
            if self.config.history_file.exists():
                with open(self.config.history_file, 'r', encoding='utf-8') as f:
                    history = json.load(f)
            else:
                history = []
            
            # 新しい実行記録を追加
            session_record = {
                "session_timestamp": datetime.now().isoformat(),
                "total_captures": len(results),
                "successful_captures": len([r for r in results if r["success"]]),
                "results": results
            }
            
            history.append(session_record)
            
            # 履歴は最新10セッションまで保持
            history = history[-10:]
            
            # ファイルに保存
            with open(self.config.history_file, 'w', encoding='utf-8') as f:
                json.dump(history, f, ensure_ascii=False, indent=2)
            
            print(f"📝 履歴保存完了: {self.config.history_file}")
            
        except Exception as e:
            print(f"⚠️ 履歴保存エラー: {e}")
    
    def show_results(self, results):
        """結果サマリーを表示"""
        print("\n📊 キャプチャ結果サマリー")
        print("=" * 50)
        
        successful = [r for r in results if r["success"]]
        failed = [r for r in results if not r["success"]]
        
        print(f"✅ 成功: {len(successful)}件")
        print(f"❌ 失敗: {len(failed)}件")
        print(f"📁 保存先: {self.config.save_dir}")
        
        if successful:
            print("\n🎯 成功したキャプチャ:")
            for result in successful:
                print(f"  ✅ {result['name']}: {result['filename']}")
        
        if failed:
            print("\n❌ 失敗したキャプチャ:")
            for result in failed:
                print(f"  ❌ {result['name']}: {result['error']}")

# システム初期化
capture_system = AutoCaptureSystem(config)
print("🔧 自動キャプチャシステム初期化完了")

✅ RPA データベース初期化完了
🔧 自動キャプチャシステム初期化完了


In [5]:
# サーバー起動確認
import requests

def check_server_status():
    """サーバーの起動状況を確認"""
    try:
        response = requests.get("http://localhost:7860", timeout=5)
        if response.status_code == 200:
            print("✅ サーバー起動中 - キャプチャ実行可能")
            return True
        else:
            print(f"⚠️ サーバー応答異常: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ サーバー未起動または接続エラー: {e}")
        print("💡 app.py を実行してからキャプチャを開始してください")
        return False

server_ready = check_server_status()

✅ サーバー起動中 - キャプチャ実行可能


In [ ]:
# 🚀 メインキャプチャ実行
if server_ready and RPA_AVAILABLE:
    print("🎬 自動キャプチャを開始します...")
    
    # 非同期でキャプチャ実行
    results = await capture_system.capture_all_urls()
    
    # 結果表示
    capture_system.show_results(results)
    
    # 履歴保存
    capture_system.save_history(results)
    
    print("\n🎉 自動キャプチャ完了！")
    
else:
    print("⚠️ キャプチャ実行条件が満たされていません")
    if not server_ready:
        print("   - サーバーが起動していません")
    if not RPA_AVAILABLE:
        print("   - RPA機能が利用できません")

In [ ]:
# キャプチャ履歴の表示
def show_capture_history():
    """キャプチャ履歴を表示"""
    try:
        if config.history_file.exists():
            with open(config.history_file, 'r', encoding='utf-8') as f:
                history = json.load(f)
            
            print("📚 キャプチャ履歴")
            print("=" * 50)
            
            for i, session in enumerate(reversed(history[-5:]), 1):
                timestamp = session["session_timestamp"][:19].replace("T", " ")
                total = session["total_captures"]
                success = session["successful_captures"]
                
                print(f"#{i} {timestamp} - {success}/{total} 成功")
                
                for result in session["results"]:
                    status = "✅" if result["success"] else "❌"
                    name = result["name"]
                    if result["success"]:
                        filename = result["filename"]
                        print(f"    {status} {name}: {filename}")
                    else:
                        error = result["error"][:50]
                        print(f"    {status} {name}: {error}")
                print()
        else:
            print("📭 履歴ファイルが見つかりません")
            
    except Exception as e:
        print(f"❌ 履歴表示エラー: {e}")

show_capture_history()

In [ ]:
# 保存されたファイル一覧表示
def show_saved_files():
    """保存されたキャプチャファイル一覧を表示"""
    try:
        files = list(config.save_dir.glob("*.png"))
        files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
        
        print(f"📁 保存ファイル一覧 ({len(files)}件)")
        print("=" * 50)
        
        for i, file in enumerate(files[:10], 1):
            stat = file.stat()
            size_mb = stat.st_size / (1024 * 1024)
            mtime = datetime.fromtimestamp(stat.st_mtime).strftime("%Y-%m-%d %H:%M:%S")
            
            print(f"{i:2d}. {file.name}")
            print(f"    📅 {mtime} | 📏 {size_mb:.2f}MB")
        
        if len(files) > 10:
            print(f"    ...他 {len(files) - 10} ファイル")
            
    except Exception as e:
        print(f"❌ ファイル一覧取得エラー: {e}")

show_saved_files()

In [ ]:
# 手動キャプチャ実行（テスト用）
async def manual_capture_test(url, name):
    """手動でキャプチャをテスト実行"""
    print(f"🧪 手動キャプチャテスト: {name}")
    
    target_info = {
        "name": name,
        "url": url,
        "filename_prefix": "manual_test",
        "wait_time": 3
    }
    
    result = await capture_system.capture_single_url(target_info)
    
    if result["success"]:
        print(f"✅ テストキャプチャ成功: {result['filename']}")
    else:
        print(f"❌ テストキャプチャ失敗: {result['error']}")
    
    return result

# 使用例（必要に応じてコメントアウトを外す）
# if server_ready and RPA_AVAILABLE:
#     test_result = await manual_capture_test("http://localhost:7860", "手動テスト")

## 🎯 使用方法

### 🚀 基本的な使い方
1. **アプリケーション起動**: `python app.py` でメインアプリを起動
2. **ノートブック実行**: 上から順番にセルを実行
3. **自動キャプチャ**: メインキャプチャセルで一括実行
4. **結果確認**: 履歴とファイル一覧で確認

### 📁 ファイル保存先
- **キャプチャ画像**: `/docs/images/auto_captures/`
- **履歴ファイル**: `/docs/images/auto_captures/capture_history.json`

### 🔧 カスタマイズ
- **対象URL追加**: `CaptureConfig` クラスの `target_urls` に追加
- **保存先変更**: `save_dir` を変更
- **待機時間調整**: `wait_time` で調整

### 💡 トラブルシューティング
- **サーバー未起動**: `python app.py` でアプリを起動
- **RPA機能エラー**: Playwright のインストール確認
- **権限エラー**: 保存先ディレクトリの権限確認